# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [4]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [3]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [5]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [6]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':')
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [9]:
document = ET.parse( './data/mondial_database.xml' )

In [10]:
country = []
mortality = []

# load data in these 2 
for element in document.iterfind('country'):
    country.append(element.find('name').text)
    element_value = element.find('infant_mortality')
    if element_value is not None:
        mortality.append(float(element_value.text))
    else:
        mortality.append(0)
      
import pandas as pd

# create pandas data-frame 
df = pd.DataFrame(mortality, index = country, columns = ['infant_mortality'])
df

,infant_mortality
Albania,13.19
Greece,4.78
Macedonia,7.90
Serbia,6.16
Montenegro,0.00
...,...
Swaziland,54.82
Reunion,7.50
Saint Helena,17.63
Sao Tome and Principe,49.16


In [19]:
# Solution#1. 10 countries with the lowest infant mortality rates
df1 = df[df.infant_mortality > 0].sort_values('infant_mortality').head(10)
df1

,infant_mortality
Monaco,1.81
Japan,2.13
Bermuda,2.48
Norway,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13
Iceland,3.15


In [40]:
# Solution#2. 10 cities with the largest population
# new dictionary to store key as city-names and value as population
city_population = {}
for element in document.iterfind('country'):
    for subelement in element.getiterator('city'):
        # only take cities that have a value for population
        if subelement.find('population') is not None:
            #key = city name
            #value = last element of the list of population returned by the 'findall'
            city_population[subelement.find('name').text] = int(subelement.findall('population')[-1].text)

#convert to data-frame with index as city-name, and columns as population
df2 = pd.DataFrame(city_population.values(), index = city_population.keys(), columns = ['latest population'])
#sort descending
df2 = df2.sort_values('latest population', ascending = False)
#return top 10
df2.head(10)
        


,latest population
Shanghai,22315474
Istanbul,13710512
Mumbai,12442373
Moskva,11979529
Beijing,11716620
São Paulo,11152344
Tianjin,11090314
Guangzhou,11071424
Delhi,11034555
Shenzhen,10358381


In [65]:
# Solution#3 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
ethnicity_population = {}
for element in document.iterfind('country'):
    # the latest or the last population value of that country
    country_population = int(element.findall('population')[-1].text)
    for each_ethnicity in element.iter('ethnicgroup'):
        percentage = float(each_ethnicity.attrib['percentage'])
        each_ethnicity_population = (percentage / 100) * country_population
        
        # The above was per country-wise. The below is for overall where we add if the ethnic group is already present in dict
        if each_ethnicity.text in ethnicity_population:
            ethnicity_population[each_ethnicity.text] = ethnicity_population[each_ethnicity.text] + each_ethnicity_population
        else:
            ethnicity_population[each_ethnicity.text] = each_ethnicity_population

#convert to data-frame with index as ethnicgroup, and columns as overall-population
df3 = pd.DataFrame(ethnicity_population.values(), index = ethnicity_population.keys(), columns = ['overall population'])
#sort descending
df3 = df3.sort_values('overall population', ascending = False)
#return top 10
df3.head(10)


,overall population
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08
Malay,1.219936e+08


In [88]:
# Solution#4 name and country of a) longest river, b) largest lake and c) airport at highest elevation
airport_elevation = {}
for element in document.iterfind('airport'):
    if element.find('elevation') is not None:
        if element.find('elevation').text is not None:
            # the value is a tuple with country and elevation
            airport_elevation[element.find('name').text] = (element.find('elevation').text, element.attrib['country'])
airport_elevation    
#convert to data-frame 
df4 = pd.DataFrame(airport_elevation.values(), index = airport_elevation.keys(), columns = ['elevation', 'country'])
#sort descending
#df4 = df4.sort_values('elevation', ascending = False)
#return top 10
#df4.head(10)
df4
